In [1]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [2]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [27]:
include("../VARs/utils.jl")
includet("../RS/hmm_est.jl")

In [53]:
df = DataFrame(XLSX.readtable("../../data/usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
start = Date(1955, 01,01)
end_d = Date(2024, 07, 28)



display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))


display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1228,1.3903,0.1011,0.3652,-2.6994,-1.1768,-0.1044,0.2218,4.9165,0.8232,0.1442
Price_10Y,0.0755,1.3969,0.0531,0.1909,-0.9866,0.3097,-0.0893,0.1022,1.9867,0.7037,0.0377


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1239,1.3604,0.0656,0.3354,-2.5776,-1.0987,-0.1294,0.2287,4.1943,0.5297,0.1107
Price_10Y_CPI,0.078,1.279,0.0177,0.1544,-1.1677,0.2268,-0.1322,0.1437,2.0153,0.2268,0.0058


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0       0.188888
 0.188888  1.0

In [54]:
assets_names = [:Total_Stock_CPI,  :Price_10Y_CPI]
n_assets = length(assets_names)

2

In [55]:
n_regimes = 2
hmm = hmm_est(real_returns, n_regimes)
regime_summary(hmm[1], assets_names, 12)


Regime,Total_Stock_CPI,Price_10Y_CPI
1,0.128,0.006
2,-0.097,0.053


Regime,Total_Stock_CPI,Price_10Y_CPI
1,0.089,0.059
2,0.181,0.113


,Total_Stock_CPI,Price_10Y_CPI
Total_Stock_CPI,1.000,0.216
Price_10Y_CPI,0.216,1.000


,Total_Stock_CPI,Price_10Y_CPI
Total_Stock_CPI,1.000,0.218
Price_10Y_CPI,0.218,1.000


Regimes transition matrix


Regime,1,2
1,0.962,0.038
2,0.112,0.888


In [56]:
scenarios = simulate_hmm(hmm[1], n_assets, 25*freq, 10_000)

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,Total_Stock_CPI,Price_10Y_CPI
1,0.0973,0.0112
5,0.0754,0.0167
10,0.0727,0.0174
25,0.071,0.018


,Total_Stock_CPI,Price_10Y_CPI
1,0.1199,0.0694
5,0.0661,0.0344
10,0.0478,0.0249
25,0.0309,0.0159


,Total_Stock_CPI,Price_10Y_CPI
1,-0.6414,0.1906
5,-0.5445,0.209
10,-0.3892,0.1767
25,-0.2343,0.124


,Total_Stock_CPI,Price_10Y_CPI
1,1.2975,0.5189
5,0.5521,0.2748
10,0.2812,0.2212
25,0.0607,0.0579


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.2594,-0.1773,-0.1087,0.03,0.1054,0.1773,0.2737,0.3071,0.3441
5,-0.1014,-0.0723,-0.0437,0.0358,0.082,0.1219,0.1724,0.1884,0.2063
10,-0.051,-0.0309,-0.011,0.0431,0.0759,0.1064,0.1449,0.1579,0.172
25,-0.0063,0.0077,0.018,0.0505,0.072,0.0925,0.1199,0.1279,0.1369


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1465,-0.1193,-0.0969,-0.0358,0.0099,0.0554,0.126,0.1538,0.1921
5,-0.0598,-0.048,-0.0381,-0.0064,0.0154,0.0387,0.0749,0.0883,0.1046
10,-0.0388,-0.0292,-0.0222,0.0005,0.0166,0.0337,0.0597,0.0687,0.0801
25,-0.0172,-0.0121,-0.0076,0.0071,0.0177,0.0287,0.0443,0.0499,0.0572


In [57]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.181802  0.289992  0.373454  0.472538  0.662773

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 24.0  45.0  67.0  103.0  208.0

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.136608  0.19597  0.244768  0.309454  0.456723

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 41.0  75.0  112.0  167.0  284.0